# Numismatic Grader

This notebook is designed for creating a TensorFlow machine learning model for grading mercury dimes.

## Imports

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from PIL import Image
import os

## Dataset

Get the dataset from the images to train TensorFlow model.

### Setup

In [ ]:
input_dir = "assets/images"
output_dir = "assets/dataset"

filenames = os.listdir(input_dir)

for filename in filenames:
  # Load image and convert to grayscale
  image = Image.open(os.path.join(input_dir, filename)).convert('L')
    
  # Resize image to 1024x1024
  image = image.resize((1024, 1024))
    
  # Save image to output directory with same filename
  image.save(os.path.join(output_dir, filename))

### Create dataset

In [ ]:
# Define function to load and preprocess image
def load_and_preprocess_image(filename):
    image = tf.io.read_file(filename)
    image = tf.image.decode_png(image, channels=1)
    image = tf.image.convert_image_dtype(image, tf.float32)
    return image

# Create dataset from output directory
dataset = tf.data.Dataset.list_files(output_dir + "/*")
dataset = dataset.map(load_and_preprocess_image)

## Training

Train the model from the dataset.

### Define Model

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(1024, 1024, 1)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])

### Compile Model

In [ ]:
model.compile(optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

### Train Model

In [ ]:
model.fit(dataset, epochs=10)